In [1]:
from llama_index.llms.together import TogetherLLM
from llama_index.core import Settings

llm = TogetherLLM(model="META-LLAMA/LLAMA-3-70B-CHAT-HF", 
api_key="87cac61702f938de91651d60977c8a2978163b011f9ba6e991f0e954b9c2c2ce")
Settings.llm = llm

/home/chandanha/Documents/My_Documents/Excel_data_reader/my_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llama_index.readers.file import PandasExcelReader
import importlib
from pathlib import Path

base_reader = PandasExcelReader()
base_docs = base_reader.load_data(Path("Tv_catalogue.xlsx"))

In [3]:
print(base_docs[0].get_content())

Samsung Samsung 43 Inches Full HD LED TV (43J5100, Black) 43 Inch LED Full HD TV (43J5100) 43J5100 37200 43 inch LED Full-HD 2 2 Black 16 W
Samsung Samsung 32 Inches HD Ready LED Smart TV (Series 4 UA32N4305ARXXL, Black) 32 Inch LED HD Ready TV (Series 4 UA32N4305ARXXL) UA32N4305ARXXL 14499 32 inch LED HD-Ready 1 2 Black 10W
Samsung Samsung 49 Inches Full HD LED Curved TV (49M6300, Grey) 49 Inch LED Full HD TV (49M6300) 49M6300 91850 49 inch LED Full-HD 2 3 Black 20 W
Samsung Samsung 43 Inches Full HD LED Smart TV (43M5570, Grey) 43 Inch LED Full HD TV (43M5570) 43M5570 33790 43 inch LED Full-HD 2 3 Black 20 W
Samsung Samsung 50 Inches Ultra HD (4K) LED Smart TV (50MU6100, Black) 50 Inch LED Ultra HD (4K) TV (50MU6100) 50MU6100 69000 50 inch LED 4K 2 3 Black 20 W
Samsung Samsung 75 Inches Ultra HD (4K) LED Smart TV (75MU7000, Black) 75 Inch LED Ultra HD (4K) TV (75MU7000) 75MU7000 599900 75 inch LED 4K 3 4 Black 40 W
Samsung Samsung 65 Inches Ultra HD (4K) LED Smart TV (65MU7500, Silve

In [4]:
from llama_index.core import SummaryIndex
base_index = SummaryIndex.from_documents([base_docs[0]])
base_query_engine = base_index.as_query_engine()

In [5]:
query_str = "What are all the different Screen Type available"
base_response = base_query_engine.query(query_str)

In [6]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
Based on the provided context information, the different screen types available are:

1. LED
2. QLED
3. OLED


In [7]:
query_str = "What was the maximum, minimum and average Price"
base_response = base_query_engine.query(query_str)

In [8]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
Based on the provided context information, the maximum price is 599900, the minimum price is 14499, and the average price is approximately 74444.


In [9]:
query_str = "List all the different types of Resolution Standard"
base_response = base_query_engine.query(query_str)

In [10]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
Based on the provided context information, the different types of Resolution Standards mentioned are:

1. Full HD
2. HD Ready
3. Ultra HD (4K)
4. QLED


In [11]:
query_str = "Which brand has the highest number of TV models listed?"
base_response = base_query_engine.query(query_str)

In [12]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
Samsung has the highest number of TV models listed, with 9 models.


In [13]:
query_str = "Which TV model has the highest speaker output RMS?"
base_response = base_query_engine.query(query_str)

In [14]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
The TV model with the highest speaker output RMS is the Sony 55 Inches Ultra HD (4K) LCD TV (KD-55X9300E, Black) with a speaker output RMS of 60 W.


In [15]:
query_str = "Explain about cricket"
base_response = base_query_engine.query(query_str)

In [16]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
I apologize, but there is no information about cricket in the provided context. The context appears to be a list of TV models from various brands, including Samsung, LG, Panasonic, and Sony, with their specifications and prices. There is no mention of cricket or any related topic.


In [17]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate

chat_text_qa_msgs = [
# Defines the system role message for the chat interaction like guidelines and tone for the AI model.
ChatMessage(
    role=MessageRole.SYSTEM,
    content=("""
        You are a AI Chatbot assistance to answer the question for the excel data i have given.
        Your goal is to answer questions as accurately as possible based on the instructions and context provided.
        After answering to the query at the end tell 'Let me Know if You Have any Queries'
        If the context is Outside the Excel date Don't answer related to the question.
        Just say that, 'The asked question is outside the context of the Excel data, I will answer to the questions related to the excel only'.
        If the context is Outside the Excel Don't try to make up an answer.
        It the question is asked about max, min, average of a particular colum go through all the rows of the column
        For general questions like 'How are you?' or 'Who are you?', respond accordingly,"""
    ),
),
# Defines the system user message for the chat interaction like query and passing relevent chunks to the AI model.
ChatMessage(
    role=MessageRole.USER,
    content=(
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information and not prior knowledge, "
        "answer the question: {query_str} in bullet points or numbered list where appropriate.\n"
    ),
),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [18]:
from llama_index.core import SummaryIndex
base_index = SummaryIndex.from_documents([base_docs[0]])
base_query_engine = base_index.as_query_engine(text_qa_template=text_qa_template, similarity_top_k=10)

In [19]:
from llama_index.core import Settings

Settings.chunk_size = 100
Settings.chunk_overlap = 20
Settings.context_window = 8100

In [20]:
query_str = "Explain about cricket"
base_response = base_query_engine.query(query_str)

In [21]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
The asked question is outside the context of the Excel data, I will answer to the questions related to the excel only. There is no information about cricket in the provided Excel data. The data appears to be related to TVs and their specifications. If you have any questions related to the TV data, I'd be happy to help! 

Let me know if you have any queries.


In [22]:
import nest_asyncio

nest_asyncio.apply()

In [23]:
from llama_index.core.evaluation import FaithfulnessEvaluator

# define evaluator
evaluator = FaithfulnessEvaluator(llm=llm)

response = base_query_engine.query(
    "Which TV model has the highest speaker output RMS?"
)
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

False


In [24]:
response

Response(response='Based on the provided context information, here is the answer to the question:\n\n* The TV model with the highest speaker output RMS is the Sony Sony 55 Inches Ultra HD (4K) LCD TV (KD-55X8200E, Black) with a speaker output RMS of 40 W.\n\nLet me know if you have any further queries!', source_nodes=[NodeWithScore(node=TextNode(id_='b7f5a66f-9c37-462d-948a-23a3e2f7d348', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3acaea83-0063-4535-a022-a0e08d105390', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='2b26050ac68b808a1d827dee75c888136682ca2bf844178afc197953617f165c'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='f6708db6-ed92-4f36-8fe2-6dc0d386519b', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='adc303e8bd74dbbfc8e560f75dede79c21b3abed92c8e87f7559d3a7749faf06')}, text='Samsung Samsung 43 Inches Full HD LED TV (43J5100, Black)

In [25]:
from llama_index.core.evaluation import FaithfulnessEvaluator

# define evaluator
evaluator = FaithfulnessEvaluator(llm=llm)

response = base_query_engine.query(
    "How many TVs have more than 2 USB ports?"
)
eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

True


In [26]:
response

Response(response='Based on the provided context information, I can answer your question as follows:\n\nHere are the TVs with more than 2 USB ports:\n\n1. Samsung 43 Inches Full HD LED Smart TV (43M5570, Grey) - 3 USB ports\n2. Samsung 50 Inches Ultra HD (4K) LED Smart TV (50MU6100, Black) - 3 USB ports\n3. Samsung 65 Inches Ultra HD (4K) LED Smart TV (65MU7500, Silver) - 4 USB ports\n4. Samsung 75 Inches Ultra HD (4K) LED Smart TV (75MU7000, Black) - 4 USB ports\n5. LG 65 Inches Ultra HD OLED Smart TV (OLED65B8PTA, Grey) - 3 USB ports\n6. LG 65 Inches Ultra HD (4K) LED Smart TV (65UK7500PTA, Silver) - 4 USB ports\n7. Panasonic 65 Inches Ultra HD (4K) LED Smart TV (TH-65FX800D, Grey) - 4 USB ports\n8. Sony 55 Inches Ultra HD (4K) LCD TV (KD-55X8200E, Black) - 4 USB ports\n9. Sony 65 Inches Ultra HD (4K) LCD TV (KD-65X9300E, Black) - 4 USB ports\n\nLet me know if you have any further queries!', source_nodes=[NodeWithScore(node=TextNode(id_='b7f5a66f-9c37-462d-948a-23a3e2f7d348', embeddi

In [27]:
query_str = "Which TV model has the most number of HDMI ports?"
base_response = base_query_engine.query(query_str)

In [28]:
print("******* PandasExcelReader *******")
print(str(base_response))

******* PandasExcelReader *******
Based on the provided context information, here is the answer to the question:

The TV models with the most number of HDMI ports are:

* Samsung 65 Inches Ultra HD (4K) LED Smart TV (65MU7500, Silver) - 4 HDMI ports
* LG LG 65 Inches Ultra HD OLED Smart TV (OLED65B8PTA, Grey) - 4 HDMI ports
* LG LG 65 Inches Ultra HD (4K) LED Smart TV (65UK7500PTA, Silver) - 4 HDMI ports
* Panasonic Panasonic 65 Inches Ultra HD (4K) LED Smart TV (TH-65FX800D, Grey) - 4 HDMI ports
* Sony Sony 55 Inches Ultra HD (4K) LED Smart Android TV (55X9000H, Black) - 4 HDMI ports

Let me know if you have any further queries!


In [30]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [31]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# build index
index = VectorStoreIndex.from_documents(base_docs)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)],
)

# query
response = query_engine.query("How can you help me")
print(response)

Empty Response


In [33]:
# %pip install llama-index
# %pip install llama-index-retrievers-bm25

In [35]:

from llama_index.core.node_parser import SentenceSplitter

# initialize node parser
splitter = SentenceSplitter(chunk_size=512)

nodes = splitter.get_nodes_from_documents(base_docs)

In [36]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    nodes=nodes,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [37]:

bm25_retriever.persist("./bm25_retriever")

loaded_bm25_retriever = BM25Retriever.from_persist_dir("./bm25_retriever")

Finding newlines for mmindex: 100%|██████████| 21.1k/21.1k [00:00<00:00, 9.01MB/s]


In [38]:
# initialize a docstore to store nodes
# also available are mongodb, redis, postgres, etc for docstores
from llama_index.core.storage.docstore import SimpleDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [39]:
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer

# We can pass in the index, docstore, or list of nodes to create the retriever
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore,
    similarity_top_k=2,
    # Optional: We can pass in the stemmer and set the language for stopwords
    # This is important for removing stopwords and stemming the query + text
    # The default is english for both
    stemmer=Stemmer.Stemmer("english"),
    language="english",
)

In [41]:
%pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached contourpy-1.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 8.5 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.2.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (306 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.4 MB/s eta 0:00:00ta 0:00:01
Using cached kiwisolver-1.4.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.4 MB)
Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)
Note: you may need to restart the kernel to use updated packages.


In [42]:
from llama_index.core.response.notebook_utils import display_source_node

# will retrieve context from specific companies
retrieved_nodes = bm25_retriever.retrieve(
    "List all the different types of Resolution Standard"
)
for node in retrieved_nodes:
    display_source_node(node, source_length=5000)

Matplotlib is building the font cache; this may take a moment.


**Node ID:** 98108259-ed51-4e3f-910d-499fb2a43850<br>**Similarity:** 0.0<br>**Text:** Black) 40 Inch LED Full HD TV (KLV-40W672E) KLV-40W672E 41499 40 inch LED Full-HD 2 2 Black 30 W
Sony Sony 65 Inches Ultra HD (4K) LCD TV (KD-65X9300E, Black) 65 Inch LED Ultra HD (4K) TV (KD-65X9300E) KD-65X9300E 179999 65 inch LED 4K 3 4 Black 60 W
Sony Sony 49 Inches Ultra HD (4K) LCD Smart TV (KD-49X7500E, Black) 49 Inch LED Ultra HD (4K) TV (KD-49X7500E) KD-49X7500E 71999 49 inch LED 4K 3 4 Black 20 W<br>

**Node ID:** 0a2c2fac-6b48-4147-b8d5-3d63da073597<br>**Similarity:** 0.0<br>**Text:** Black) 55 Inch LED Ultra HD (4K) TV (55UK6360PTE) 55UK6360PTE 58999 55 inch LED 4K 1 2 Black 20 W
LG LG 65 Inches Ultra HD OLED Smart TV (OLED65B8PTA, Grey) 65 Inch OLED Ultra HD (4K) TV (OLED65B8PTA) OLED65B8PTA 259350 65 inch OLED 4K 1 3 Black 40 W
LG LG 32 Inches HD Ready LED Smart TV (32LK628BPTF) 32 Inch LED HD Ready TV (32LK628BPTF) 32LK628BPTF 20990 32 inch LED HD-Ready 1 3 Black 35 W
LG LG 55 Inches Ultra HD LED Smart TV (55UK6500PTC, Black) 55 Inch LED Ultra HD (4K) TV (55UK6500PTC) 55UK6500PTC 71462 55 inch LED 4K 2 4 Black 20 W
LG LG 55 Inches Ultra HD (4K) LED Smart TV (55UK7500PTA, Silver) 55 Inch LED Ultra HD (4K) TV (55UK7500PTA) 55UK7500PTA 65195 55 inch LED 4K 2 4 Black 20 W
LG LG 55 Inches Ultra HD (4K) OLED Smart TV (OLED55B8PTA, Grey) 55 Inch OLED Ultra HD (4K) TV (OLED55B8PTA) OLED55B8PTA 159999 55 inch OLED 4K 1 3 Grey 40 W
LG LG 65 Inches Ultra HD (4K) OLED Smart TV (OLED65C8PTA) 65 Inch OLED Ultra HD (4K) TV (OLED65C8PTA) OLED65C8PTA 249999 65 inch OLED 4K 1 2 Black 40 W
LG LG 65 Inches Ultra HD (4K) LED Smart TV (65UK7500PTA,<br>

In [43]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import PydanticSingleSelector
from llama_index.core.tools import QueryEngineTool


list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description="Useful for summarization questions related to the data source",
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context related to the data source",
)

query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)
query_engine.query("<query>")

NameError: name 'list_query_engine' is not defined

In [44]:
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

# pydantic selectors feed in pydantic objects to a function calling API
# single selector (pydantic)
selector = PydanticSingleSelector.from_defaults()
# multi selector (pydantic)
selector = PydanticMultiSelector.from_defaults()

# LLM selectors use text completion endpoints
# single selector (LLM)
selector = LLMSingleSelector.from_defaults()
# multi selector (LLM)
selector = LLMMultiSelector.from_defaults()

ValueError: Model name META-LLAMA/LLAMA-3-70B-CHAT-HF does not support function calling API. 